In [ ]:
#| default_exp core

In [ ]:
#| hide
from nbdev.showdoc import *

In [ ]:
#| export
import gymnasium as gym
from gymnasium import spaces
import numpy as np
import pandas as pd
from typing import Tuple, Optional, Union


class CryptoTradingEnv(gym.Env):
    """
    A cryptocurrency trading environment for reinforcement learning.

    This environment simulates trading a single cryptocurrency based on historical price data.
    It supports buying, selling, and holding actions, and provides observations 
    that can include price history, technical indicators, and the agent's current holdings.

    Key Concepts:
        -   Action Space:  Discrete (buy, sell, hold).  Could be extended to continuous.
        -   Observation Space: Box (multidimensional array).  Shape depends on features.
        -   Reward Function:  Change in portfolio value (can be customized).
        -   Episode Termination:  End of data or when the agent runs out of funds.
        -   Data: Requires a pandas DataFrame with 'close' prices (and optionally other features).

    """

    metadata = {"render_modes": ["human"], "render_fps": 30}  # For future rendering capabilities

    def __init__(
        self,
        data: pd.DataFrame,
        initial_balance: float = 10000.0,
        transaction_fee_percent: float = 0.001,  # 0.1% transaction fee
        lookback_window_size: int = 50,  # Number of past timesteps to include in observation
        features: list[str] = ["close"],  # List of columns to use as features
        render_mode: Optional[str] = None,
    ) -> None:
        """
        Initializes the CryptoTradingEnv.

        Args:
            data (pd.DataFrame):  Historical price data (and other features).  MUST include a 'close' column.
            initial_balance (float): Starting capital for the agent.
            transaction_fee_percent (float):  Fee percentage per transaction (e.g., 0.001 for 0.1%).
            lookback_window_size (int):  How many past timesteps to use in the observation.
            features (List[str]):  Column names from 'data' to use as observation features.
            render_mode (str, optional): Render mode for human visualization. Defaults to None.
        Raises:
            ValueError: If input data is invalid or required columns are missing.
        """
        super().__init__()

        if not isinstance(data, pd.DataFrame):
            raise ValueError("Data must be a pandas DataFrame.")
        if "close" not in data.columns:
            raise ValueError("Data must contain a 'close' column for price data.")
        if not all(feature in data.columns for feature in features):
            raise ValueError("All features must be present in the data columns.")
        if lookback_window_size <= 0:
            raise ValueError("lookback_window_size must be greater than 0.")

        self.data = data
        self.features = features
        self.initial_balance = initial_balance
        self.transaction_fee_percent = transaction_fee_percent
        self.lookback_window_size = lookback_window_size
        self.render_mode = render_mode

        # Action Space: 0 = Hold, 1 = Buy, 2 = Sell
        self.action_space = spaces.Discrete(3)

        # Observation Space: (lookback_window_size, num_features) + balance + holdings
        self.observation_space = spaces.Box(
            low=-np.inf,
            high=np.inf,
            shape=(lookback_window_size, len(self.features) + 2),
            dtype=np.float32,
        )

        self.current_step = lookback_window_size
        self.balance = initial_balance
        self.holdings = 0.0  # Amount of crypto held
        self.portfolio_value = initial_balance  # balance + (holdings * current_price)
        self._done = False  # added to track done state explicitly

    def _get_observation(self) -> np.ndarray:
        """
        Constructs the observation array.

        Returns:
            np.ndarray:  The observation array.
        """
        window_data = self.data[self.features][
            self.current_step - self.lookback_window_size : self.current_step
        ].values

        # Add balance and holdings to the observation.  Reshape for consistent shape.
        balance_and_holdings = np.array([[self.balance, self.holdings]])
        repeated_balance_holdings = np.repeat(
            balance_and_holdings, self.lookback_window_size, axis=0
        )
        observation = np.concatenate(
            (window_data, repeated_balance_holdings), axis=1
        )

        return observation.astype(np.float32)

    def _calculate_reward(self, action: int) -> float:
        """
        Calculates the reward based on the change in portfolio value.

        Args:
            action (int): The action taken by the agent.

        Returns:
            float: The reward for the current step.  (Can be customized).
        """
        current_price = self.data["close"][self.current_step]
        previous_portfolio_value = self.portfolio_value
        self.portfolio_value = self.balance + (self.holdings * current_price)
        reward = self.portfolio_value - previous_portfolio_value
        return reward

    def _execute_trade(self, action: int) -> None:
        """
        Executes a trade (buy or sell) and updates balance/holdings.

        Args:
            action (int): The action taken (1 for buy, 2 for sell).
        """

        current_price = self.data["close"][self.current_step]

        if action == 1:  # Buy
            affordable_amount = self.balance / current_price
            cost = affordable_amount * current_price
            fee = cost * self.transaction_fee_percent
            if self.balance >= cost + fee:
                self.balance -= cost + fee
                self.holdings += affordable_amount * (1-self.transaction_fee_percent)

        elif action == 2:  # Sell
            fee = self.holdings * current_price * self.transaction_fee_percent
            self.balance += self.holdings * current_price - fee
            self.holdings = 0.0

    def step(
        self, action: int
    ) -> Tuple[np.ndarray, float, bool, bool, dict]:
        """
        Takes a step in the environment.

        Args:
            action (int): The action to take (0, 1, or 2).

        Returns:
            Tuple: A tuple containing:
                - observation (np.ndarray): The next state's observation.
                - reward (float): The reward for the action.
                - terminated (bool): Whether the episode has ended.
                - truncated (bool): Whether the episode was truncated (not used here, but required by gym API).
                - info (dict): Additional information (empty in this case).
        """
        if self._done:
            # If the episode is already done, you can either reset it or raise an error.
            # Here we reset.  Alternatively, you can raise:  raise Exception("Episode already done")
            return self.reset()[0], 0.0, True, False, {}
        
        self._execute_trade(action)  # Execute the trade first
        reward = self._calculate_reward(action)  # Calculate reward *after* trade

        self.current_step += 1
        # Check if the episode is done (either end of data or out of money)
        self._done = (
            self.current_step >= len(self.data) - 1
            or self.balance <= 0
        )
        truncated = False
        observation = self._get_observation()
        info: dict = {}
        return observation, reward, self._done, truncated, info  # type: ignore

    def reset(
        self, *, seed: Optional[int] = None, options: Optional[dict] = None
    ) -> Tuple[np.ndarray, dict]:
        """
        Resets the environment to the initial state.

        Args:
            seed (Optional[int]): Seed for random number generation.
            options (Optional[dict]):  Additional options (not used here).

        Returns:
            Tuple: A tuple containing:
                - observation (np.ndarray): The initial observation.
                - info (dict):  Additional info (empty in this case).
        """
        super().reset(seed=seed)  # Call gym.Env's reset with the seed

        self.current_step = self.lookback_window_size
        self.balance = self.initial_balance
        self.holdings = 0.0
        self.portfolio_value = self.initial_balance
        self._done = False  # Reset the done flag
        observation = self._get_observation()
        info: dict = {}
        return observation, info

    def render(self) -> None:
        """
        Renders the environment (not implemented here, but could be used for visualization).
        """
        # Future implementation for visualization (e.g., using matplotlib).
        if self.render_mode == "human":
            print(
                f"Step: {self.current_step}, Balance: {self.balance:.2f}, "
                f"Holdings: {self.holdings:.4f}, Portfolio Value: {self.portfolio_value:.2f}"
            )


# Example Usage (and testing)
if __name__ == "__main__":
    # Create sample data (replace with your actual data)
    num_days = 200
    data = pd.DataFrame(
        {
            "close": np.random.rand(num_days) * 100 + 100,  # Example prices
            "volume": np.random.rand(num_days) * 1000 + 500,  # Example volume
        }
    )
    data["returns"] = data["close"].pct_change()

    # Create and use the environment
    env = CryptoTradingEnv(
        data=data,
        initial_balance=1000,
        transaction_fee_percent=0.001,
        lookback_window_size=20,
        features=["close", "volume", "returns"],
    )

    # Basic interaction loop
    obs, _ = env.reset()
    n_steps = 50
    for _ in range(n_steps):
        action = env.action_space.sample()  # Replace with your agent's action
        obs, reward, terminated, truncated, info = env.step(action)
        done = terminated or truncated
        env.render()
        if done:
            print("Episode finished")
            obs, _ = env.reset()  # Reset if the episode is finished
    print("Interaction loop finished.")

In [ ]:
#| hide
import nbdev; nbdev.nbdev_export()